In [1]:
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import chi2_contingency 
from pipelines import preprocessor, preprocessor2
from preprocess import extra_features, separate
import pickle
import joblib

In [2]:
train= pd.read_csv("Datasets/train.csv")
train=extra_features(train)
separate(train)

<IPython.core.display.Javascript object>

Target Column: ['cost_category']

Numerical Columns: ['night_mainland', 'night_zanzibar', 'total_nights']

Categorical Columns: ['age_group', 'travel_with', 'tour_arrangement', 'package_transport_int', 'package_accomodation', 'package_food', 'package_transport_tz', 'package_sightseeing', 'package_guided_tour', 'package_insurance', 'first_trip_tz', 'trip_length', 'group_size']

Text Columns: ['country', 'purpose', 'main_activity', 'info_source']


In [3]:
X=train.drop('cost_category', axis=1)
y=train['cost_category']

In [4]:
X.shape

(18506, 23)

In [5]:
X.head(2)

,country,age_group,travel_with,total_female,total_male,purpose,main_activity,info_source,tour_arrangement,package_transport_int,...,package_sightseeing,package_guided_tour,package_insurance,night_mainland,night_zanzibar,first_trip_tz,total_nights,trip_length,total_people,group_size
Tour_ID,,,,,,,,,,,,,,,,,,,,,
tour_id1hffseyw,ITALY,45-64,With Children,0.0,2.0,Visiting Friends and Relatives,Beach Tourism,"Friends, relatives",Package Tour,Yes,...,No,No,No,0,7,Yes,7,Normal,2.0,small
tour_idnacd7zag,UNITED KINGDOM,25-44,With Spouse,1.0,1.0,Leisure and Holidays,Wildlife Tourism,"Travel agent, tour operator",Package Tour,Yes,...,No,No,No,0,7,Yes,7,Normal,2.0,small


In [6]:
col_drop=[]

for col in X.columns:
    
    ct=pd.crosstab(y, X[col])
    chi2, P, _ , _ = chi2_contingency(ct)
    
    if P >= 0.05:
        col_drop.append(col)
    elif chi2 <= 1500:
            col_drop.append(col)
        
    print(f"Column: {col}\nChi_Test: {chi2}\nP_Value: {P}\n*************")

<IPython.core.display.Javascript object>

Column: country
Chi_Test: 9779.033678192025
P_Value: 0.0
*************


<IPython.core.display.Javascript object>

Column: age_group
Chi_Test: 1803.196457648891
P_Value: 0.0
*************


<IPython.core.display.Javascript object>

...

Column: trip_length
Chi_Test: 402.40609235137765
P_Value: 2.159271242364782e-76
*************


<IPython.core.display.Javascript object>

Column: total_people
Chi_Test: 5188.487023293924
P_Value: 0.0
*************


<IPython.core.display.Javascript object>

Column: group_size
Chi_Test: 686.1389145094461
P_Value: 1.6709403090257822e-136
*************


In [7]:
col_drop

['total_male', 'trip_length', 'group_size']

In [8]:
X.drop(columns=col_drop, inplace=True)
X.shape

(18506, 20)

In [9]:
X= preprocessor(X)
X

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [10]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y=le.fit_transform(y)
mapping= dict(zip(le.classes_, le.transform(le.classes_)))
mapping

{'High Cost': 0,
 'Higher Cost': 1,
 'Highest Cost': 2,
 'Low Cost': 3,
 'Lower Cost': 4,
 'Normal Cost': 5}

In [11]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, shuffle=True, stratify= y, random_state=1)

print(f"X_train: {X_train.shape}\nX_test: {X_test.shape}\ny_train: {y_train.shape}\ny_test: {y_test.shape}")

X_train: (14804, 187)
X_test: (3702, 187)
y_train: (14804,)
y_test: (3702,)


In [12]:
n_features= X.shape[1]
num_classes= len(le.classes_)

model= Sequential()
model.add(Dense(32, input_dim=n_features, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

epochs=30
learning_rate= 0.01
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                6016      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 128)               8

In [13]:
model.fit(X_train, y_train, epochs= epochs, batch_size=64, verbose=1)

Epoch 1/30
232/232 [==============================] - 37s 49ms/step - loss: 1.3337 - accuracy: 0.4457
Epoch 2/30
232/232 [==============================] - 8s 36ms/step - loss: 1.2089 - accuracy: 0.5036
Epoch 3/30
232/232 [==============================] - 8s 35ms/step - loss: 1.1755 - accuracy: 0.5378
Epoch 4/30
232/232 [==============================] - 10s 42ms/step - loss: 1.1639 - accuracy: 0.5419
Epoch 5/30
232/232 [==============================] - 9s 39ms/step - loss: 1.1497 - accuracy: 0.5525
Epoch 6/30
232/232 [==============================] - 7s 32ms/step - loss: 1.1411 - accuracy: 0.5546
Epoch 7/30
232/232 [==============================] - 8s 36ms/step - loss: 1.1318 - accuracy: 0.5597
Epoch 8/30
232/232 [==============================] - 17s 75ms/step - loss: 1.1324 - accuracy: 0.5588
Epoch 9/30
232/232 [==============================] - 10s 42ms/step - loss: 1.1246 - accuracy: 0.5663
Epoch 10/30
232/232 [==============================] - 9s 39ms/step - loss: 1.1171 - ac

In [14]:
loss, accuracy= model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy Score: {accuracy }\nLoss: {loss}")

116/116 [==============================] - 2s 11ms/step - loss: 1.1371 - accuracy: 0.5559
Accuracy Score: 0.5559157133102417
Loss: 1.1371111869812012


In [15]:
folds= StratifiedKFold(n_splits=10, shuffle=True, random_state=42 )

for fold, (train_idx, val_idx) in enumerate (folds.split(X,y)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]

In [16]:
model.fit(X_train, y_train, epochs=epochs, batch_size=64, verbose=1)

Epoch 1/30
261/261 [==============================] - 10s 36ms/step - loss: 1.0868 - accuracy: 0.5815
Epoch 2/30
261/261 [==============================] - 10s 39ms/step - loss: 1.0805 - accuracy: 0.5862
Epoch 3/30
261/261 [==============================] - 10s 37ms/step - loss: 1.0784 - accuracy: 0.5875
Epoch 4/30
261/261 [==============================] - 10s 40ms/step - loss: 1.0803 - accuracy: 0.5860
Epoch 5/30
261/261 [==============================] - 10s 38ms/step - loss: 1.0753 - accuracy: 0.5896
Epoch 6/30
261/261 [==============================] - 10s 38ms/step - loss: 1.0762 - accuracy: 0.5873
Epoch 7/30
261/261 [==============================] - 10s 36ms/step - loss: 1.0732 - accuracy: 0.5889
Epoch 8/30
261/261 [==============================] - 9s 36ms/step - loss: 1.0745 - accuracy: 0.5881
Epoch 9/30
261/261 [==============================] - 9s 35ms/step - loss: 1.0760 - accuracy: 0.5875
Epoch 10/30
261/261 [==============================] - 9s 36ms/step - loss: 1.0716 -

In [17]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy Score: {accuracy }\nLoss: {loss}")

58/58 [==============================] - 1s 10ms/step - loss: 1.0932 - accuracy: 0.5768
Accuracy Score: 0.5767567753791809
Loss: 1.0932362079620361
